<a href="https://colab.research.google.com/github/HwangSiyeon/ESAA_homework/blob/main/ESAA_HW_0412_DocumentClustering_MachineLearningGuide_516_528_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7. 문서 군집화 소개와 실습
### 문서 군집화 개념
- 문서 군집화 : 비슷한 텍스트 구성의 문서를 군집화하는 것이다. 동일 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있다.

### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기


In [8]:
import pandas as pd
import glob, os

# 다음은 저자의 컴퓨터에서 압축 파일을 풀어놓은 디렉터리이니, 각자 디렉터리를 다시 설정한다.
path =r'/content/drive/MyDrive/ESAA_OB_HW/DATA/topics'
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list =[]
opinion_text =[]

# 개별 파일의 파일명은 filename_list로 취합
# 개별 파일의 파일 내용은 DataFrame로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
  # 개별 파일을 읽어서 DataFrame으로 생성
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
  # 절대 경로로 주어진 파일명을 가공. 리눅스에서 수행할 때는 다음\\를 /로 변경
  # 맨 마지막 .data 확장자도 제거
  filename_ = file_.split('/')[-1]
  filename = filename_.split('.')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
  filename_list.append(filename)
  opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,features_windows7,...
2,voice_garmin_nuvi_255W_gps,...
3,battery-life_netbook_1005ha,...
4,screen_ipod_nano_8gb,...


- 각 파일 이름으로 의견의 텍스트가 어떤 제품/서비스에 대한 리뷰인지 알 수 있다.
- TF-IDF형태로 피처 벡터화한다. min_df와 max_df 범위 설정으로 피처개수를 제한한다. TfidVectorizer의 fit_transform()의 인자로 document_df DataFrame의 opinion_text칼럼을 입력하면 개별 문서 텍스트에 대해 TF-IDF변환된 피처 벡터화된 행렬을 구할 수 있다.


In [10]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vect = TfidfVectorizer(tokenizer = LemNormalize, stop_words='english',
                            ngram_range=(1,2), min_df = 0.05, max_df = 0.85)
# opinion_text칼럼 값으로 피처 벡터화 수행
feature_vect = tfid_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


- 문서별 텍스트가 TF-IDF변환된 피처 벡터화 행렬 데이터에 대해 k-means 군집화를 수행해 어떤 문서끼리 군집되는지 확인해보자.

In [12]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters = 5, max_iter = 10000, random_state = 0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [13]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,1
1,features_windows7,...,0
2,voice_garmin_nuvi_255W_gps,...,4
3,battery-life_netbook_1005ha,...,1
4,screen_ipod_nano_8gb,...,1


- 이제 각 클러스터별로 어떤 내용의 글들이 분포하는지 보자

In [16]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
23,buttons_amazon_kindle,...,0
32,eyesight-issues_amazon_kindle,...,0
1,features_windows7,...,0
19,fonts_amazon_kindle,...,0
27,navigation_amazon_kindle,...,0
25,price_amazon_kindle,...,0
17,speed_windows7,...,0


In [17]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
20,battery-life_amazon_kindle,...,1
0,battery-life_ipod_nano_8gb,...,1
3,battery-life_netbook_1005ha,...,1
13,keyboard_netbook_1005ha,...,1
21,performance_netbook_1005ha,...,1
4,screen_ipod_nano_8gb,...,1
16,screen_netbook_1005ha,...,1
15,size_asus_netbook_1005ha,...,1
11,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
7,video_ipod_nano_8gb,...,1


In [18]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
40,bathroom_bestwestern_hotel_sfo,...,2
35,food_holiday_inn_london,...,2
30,food_swissotel_chicago,...,2
42,free_bestwestern_hotel_sfo,...,2
37,location_bestwestern_hotel_sfo,...,2
22,location_holiday_inn_london,...,2
41,parking_bestwestern_hotel_sfo,...,2
33,price_holiday_inn_london,...,2
34,room_holiday_inn_london,...,2
43,rooms_bestwestern_hotel_sfo,...,2


In [19]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
44,comfort_honda_accord_2008,...,3
24,comfort_toyota_camry_2007,...,3
18,gas_mileage_toyota_camry_2007,...,3
48,interior_honda_accord_2008,...,3
28,interior_toyota_camry_2007,...,3
47,mileage_honda_accord_2008,...,3
45,performance_honda_accord_2008,...,3
39,quality_toyota_camry_2007,...,3
50,seats_honda_accord_2008,...,3
46,transmission_toyota_camry_2007,...,3


In [20]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
8,accuracy_garmin_nuvi_255W_gps,...,4
14,directions_garmin_nuvi_255W_gps,...,4
5,display_garmin_nuvi_255W_gps,...,4
12,satellite_garmin_nuvi_255W_gps,...,4
10,screen_garmin_nuvi_255W_gps,...,4
6,speed_garmin_nuvi_255W_gps,...,4
9,updates_garmin_nuvi_255W_gps,...,4
2,voice_garmin_nuvi_255W_gps,...,4


- 전반적으로 군집화된 결과를 살펴보면 군집 개수가 약간 많이 설정되어 있어 세분화되어 군집화된 경향이 있다. 중심개수를 5개에서 3개로 낮춰서 3개 그룹으로 군집화해보자

In [22]:
# 3개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters = 3, max_iter = 10000, random_state = 0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 소속 군집을 cluster_label칼럼으로 할당하고 cluster_label값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
50,seats_honda_accord_2008,...,0
48,interior_honda_accord_2008,...,0
47,mileage_honda_accord_2008,...,0
46,transmission_toyota_camry_2007,...,0
45,performance_honda_accord_2008,...,0
44,comfort_honda_accord_2008,...,0
24,comfort_toyota_camry_2007,...,0
28,interior_toyota_camry_2007,...,0
39,quality_toyota_camry_2007,...,0
18,gas_mileage_toyota_camry_2007,...,0


In [23]:
document_df[document_df['cluster_label']==0].sort_values(by='filename') ## 자동차 리뷰

,filename,opinion_text,cluster_label
44,comfort_honda_accord_2008,...,0
24,comfort_toyota_camry_2007,...,0
18,gas_mileage_toyota_camry_2007,...,0
48,interior_honda_accord_2008,...,0
28,interior_toyota_camry_2007,...,0
47,mileage_honda_accord_2008,...,0
45,performance_honda_accord_2008,...,0
39,quality_toyota_camry_2007,...,0
50,seats_honda_accord_2008,...,0
46,transmission_toyota_camry_2007,...,0


In [24]:
document_df[document_df['cluster_label']==1].sort_values(by='filename') ## 호텔 리뷰

,filename,opinion_text,cluster_label
40,bathroom_bestwestern_hotel_sfo,...,1
35,food_holiday_inn_london,...,1
30,food_swissotel_chicago,...,1
42,free_bestwestern_hotel_sfo,...,1
37,location_bestwestern_hotel_sfo,...,1
22,location_holiday_inn_london,...,1
41,parking_bestwestern_hotel_sfo,...,1
33,price_holiday_inn_london,...,1
34,room_holiday_inn_london,...,1
43,rooms_bestwestern_hotel_sfo,...,1


In [25]:
document_df[document_df['cluster_label']==2].sort_values(by='filename') ## 전자기기 리뷰

,filename,opinion_text,cluster_label
8,accuracy_garmin_nuvi_255W_gps,...,2
20,battery-life_amazon_kindle,...,2
0,battery-life_ipod_nano_8gb,...,2
3,battery-life_netbook_1005ha,...,2
23,buttons_amazon_kindle,...,2
14,directions_garmin_nuvi_255W_gps,...,2
5,display_garmin_nuvi_255W_gps,...,2
32,eyesight-issues_amazon_kindle,...,2
1,features_windows7,...,2
19,fonts_amazon_kindle,...,2


### 군집별 핵심 단어 추출하기
- 각 군집에 속한 문서는 핵심 단어를 주축으로 군집화되어 있다. 이번에는 각 군집을 구성하는 핵심단어가 어떤 것이 있는지 확딘해보자
- KMeeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심을 기주으로 얼마나 가깝게 위치해 있는지 cluster_centers_라는 속성으로 제공한다.
- cluster_centers_는 배열 값으로 제공되해 행은 개별 군집을, 열은 개별 피처를 나타낸다. 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫자값으로 표현한 일종의 좌표값이다.
- 예를 들어 cluster_centers[0,1]은 0번 군집에서 두번째 피처의 위치값이다.

In [29]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


- 0 ~ 1 사이에 존재함다. 이 값이 1에 가까울수록 군집과 가까운 값을 가진다.
- 이제 cluster_centers_속성값을 이용해 각 군집별 핵심 단어를 찾아보자.

In [30]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):

    cluster_details = {}

    # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰순으로 값을 구하기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_ .argsort()[:, ::-1] # 행별(군집별)로 역순 정렬

    #개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for clusters_num in range(clusters_num):
        # 군집별 정보를 담을 데이터 초기화
        cluster_details[clusters_num] = {}
        cluster_details[clusters_num]['cluster']=clusters_num

        # cluster_centers_.argsort()[:,::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
        top_feature_indexes=centroid_feature_ordered_ind[clusters_num, :top_n_features]
        top_features=[feature_names[ind] for ind in top_feature_indexes]

        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values=cluster_model.cluster_centers_[clusters_num, top_feature_indexes].tolist()

        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심 위치 상댓값, 해당 파일명 입력
        cluster_details[clusters_num]['top_features']=top_features
        cluster_details[clusters_num]['top_features_value']=top_feature_values
        filenames=cluster_data[cluster_data['cluster_label']==clusters_num]['filename']
        filenames=filenames.values.tolist()

        cluster_details[clusters_num]['filenames']=filenames

    return cluster_details

- dictionary를 원소로 가지는 리스트인 cluster_details를 반환한다. 이 cluster_details에는 개별 군집번호, 핵심단어, 핵심단어 중심위치 상댓값, 파일명 속성 값 정보가 있는데, 가독성을 위해 print_cluster_details()함수를 만든다

In [32]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features: ', cluster_detail['top_features'])
    print('Reviews 파일명: ',cluster_detail['filenames'][:7])
    print('===============================================')

In [36]:
feature_names = tfid_vect.get_feature_names_out()
cluster_details=get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                    feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features:  ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명:  ['gas_mileage_toyota_camry_2007', 'comfort_toyota_camry_2007', 'interior_toyota_camry_2007', 'quality_toyota_camry_2007', 'comfort_honda_accord_2008', 'performance_honda_accord_2008', 'transmission_toyota_camry_2007']
###### Cluster 1
Top features:  ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명:  ['location_holiday_inn_london', 'service_holiday_inn_london', 'service_swissotel_hotel_chicago', 'food_swissotel_chicago', 'staff_swissotel_chicago', 'price_holiday_inn_london', 'room_holiday_inn_london']
###### Cluster 2
Top features:  ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명:  ['battery-life_ipod_nano_8gb', 'features_windows7', 'voice_garmin_nuvi_255W_gps', 'battery-life_netbook_1005ha', 's

- 군집별 핵심단어를 알 수 있다.